## ONNX_to_TensorRT

This part of the code requires an onnx parser to convert the onnx export to tensorrt engine. Please contact me if your interested further reagrding the onnx parser to run in jetson xavier. Kindly refer to this repo for https://github.com/arvcode/TensorRT_classifier_efficientNet for reference parser that was used for Jetson Nano

In [2]:
#Import packages
import numpy as np
from onnx_helper import ONNXClassifierWrapper,convert_onnx_to_engine
import torch
import json
from PIL import Image
from torchvision import transforms

In [3]:
gpuid = 0
device = torch.device('cpu') if gpuid < 0 else torch.device(f'cuda:{gpuid}')
print('Using device:', device)

Using device: cuda:0


## For liftsplat shoot

In [4]:
ONNX_PATH='models/simp_camenc_b0_1.onnx'
TRT_PATH='models/simp_camenc_b0_1.trt'
trt_engine=convert_onnx_to_engine(ONNX_PATH, TRT_PATH)

Parsing ONNX file.
Building TensorRT engine. This may take a few minutes.


In [5]:
ONNX_PATH='models/simp_camenc_b0.onnx'
TRT_PATH='models/simp_camenc_b0.trt'
trt_engine=convert_onnx_to_engine(ONNX_PATH, TRT_PATH)

Parsing ONNX file.
Building TensorRT engine. This may take a few minutes.


## Reference


In [9]:

# logger to capture errors, warnings, and other information during the build and inference phases

TRT_LOGGER = trt.Logger()
ONNX_PATH='models/enpoints.onnx'
ONNX_PATH='models/cam_encode_b1.onnx'
#ONNX_PATH='models/efficientnetb2_batch1.onnx'
TRT_PATH='models/enpoints.trt'

def convert_onnx_to_engine(onnx_filename, engine_filename = None, max_batch_size = 32, max_workspace_size = 1 << 30, fp16_mode = True):
    logger = trt.Logger(trt.Logger.WARNING)
#https://github.com/onnx/onnx-tensorrt    
    explicit_batch = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
    
    with trt.Builder(logger) as builder, builder.create_network(explicit_batch) as network, trt.OnnxParser(network, logger) as parser:
        builder.max_workspace_size = max_workspace_size
        builder.fp16_mode = fp16_mode
        builder.max_batch_size = max_batch_size

        print("Parsing ONNX file.")
        with open(onnx_filename, 'rb') as model:
            if not parser.parse(model.read()):
                for error in range(parser.num_errors):
                    print(parser.get_error(error))

        last_layer = network.get_layer(network.num_layers - 1)
        # Check if last layer recognizes it's output
        if not last_layer.get_output(0):
            # If not, then mark the output using TensorRT API
            network.mark_output(last_layer.get_output(0))
        print("Network Input shape", network.num_inputs)
        print("Network layers", network.num_layers)
        print("Network Output shape",network.num_outputs)
        print("Building TensorRT engine. This may take a few minutes.")
        engine = builder.build_cuda_engine(network)

        if engine_filename:
            with open(engine_filename, 'wb') as f:
                f.write(engine.serialize())

        return engine, logger
